## Add project root

In [ ]:
import sys
from pathlib import Path
import importlib

# Add the parent of the *outer* DLC-Jupyter-Notebooks folder to sys.path
project_root = Path().resolve().parents[0]  # This is /Users/atanugiri/Downloads/DLC-Jupyter-Notebooks
print(project_root)
sys.path.append(str(project_root))


## Connect to db

In [ ]:
import psycopg2
import platform

host = "localhost" if platform.system() == "Windows" else "129.108.49.49"
conn = psycopg2.connect(
    dbname="deeplabcut_db", user="postgres", 
    password="1234", host=host, port="5432")
cursor = conn.cursor()


## Fetch id list

In [ ]:
import pandas as pd

task = 'FoodOnlyWhite'

saline_q = f"""
SELECT id FROM dlc_table
WHERE genotype='white' AND task='FoodOnly' AND health = 'saline' AND head_x_norm IS NOT NULL
ORDER BY id;
"""
df = pd.read_sql_query(saline_q, conn)
saline_id = df['id'].to_list()
# IDs to remove
bad_id = [549, 559, 566, 567, 570, 571, 595, 617, 621, 638, 640, 36]
# Filter them out
saline_id = [i for i in saline_id if i not in bad_id]

ghrelin_q = f"""
SELECT id FROM dlc_table
WHERE genotype='white' AND task='FoodOnly' AND health = 'ghrelin' AND head_x_norm IS NOT NULL
ORDER BY id;
"""
df = pd.read_sql_query(ghrelin_q, conn)
ghrelin_id = df['id'].to_list()
# Filter them out
ghrelin_id = [i for i in ghrelin_id if i not in bad_id]

Exc_q = f"""
SELECT id FROM dlc_table
WHERE genotype='white' AND task='FoodOnlyExcitatory' AND head_x_norm IS NOT NULL
ORDER BY id;
"""
df = pd.read_sql_query(Exc_q, conn)
Exc_id = df['id'].to_list()
Exc_id = [i for i in Exc_id if i not in bad_id]

Inh_q = f"""
SELECT id FROM dlc_table
WHERE genotype='white' AND task='FoodOnlyInhibitory' AND head_x_norm IS NOT NULL
ORDER BY id;
"""
df = pd.read_sql_query(Inh_q, conn)
Inh_id = df['id'].to_list()
Inh_id = [i for i in Inh_id if i not in bad_id]

print(f"saline_id: {saline_id}\n")
print(f"ghrelin_id: {ghrelin_id}\n")
print(f"Exc_id: {Exc_id}\n")
print(f"Inh_id: {Inh_id}\n")


## Calculate radial distances for each trajectory points

In [ ]:
import importlib
import Python_scripts.Feature_functions.radial_distances
import Python_scripts.Data_analysis.compare_distributions

importlib.reload(Python_scripts.Feature_functions.radial_distances)
importlib.reload(Python_scripts.Data_analysis.compare_distributions)

from Python_scripts.Feature_functions.radial_distances import (
    get_radial_distances, get_batch_radial_distances
)
from Python_scripts.Data_analysis.compare_distributions import compare_distributions


### Batch call with different radius_limit

In [ ]:
import numpy as np
import os
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt

pdf_path = f'{task}_radial_cdf_all_radius.pdf'
with PdfPages(pdf_path) as pdf:
    for radius in np.arange(0.1, 1.7, 0.2):
        print(f"[INFO] Analyzing radius = {radius:.1f}")

        r_saline = get_batch_radial_distances(saline_id, conn, radius_limit = radius, max_time=None)
        r_ghrelin = get_batch_radial_distances(ghrelin_id, conn, radius_limit = radius, max_time=None)
        r_exc = get_batch_radial_distances(Exc_id, conn, radius_limit = radius, max_time=None)
        r_inh = get_batch_radial_distances(Inh_id, conn, radius_limit = radius, max_time=None)
        
        if len(r_saline) == 0 or len(r_ghrelin) == 0:
            print(f"[WARNING] Skipping radius = {radius} due to missing data")
            continue

        r_saline_all = np.concatenate(r_saline)
        r_ghrelin_all = np.concatenate(r_ghrelin)
        r_exc_all = np.concatenate(r_exc)
        r_inh_all = np.concatenate(r_inh)

        fig, ax = compare_distributions(
            r_saline_all, r_ghrelin_all, r_exc_all, r_inh_all, 
            kind='cdf', return_data=False
        )
        
        ax.legend(['Saline', 'Ghrelin', 'Excitatory', 'Inhibitory'])
        ax.set_title(f"{task} | radius = {radius:.1f}")
        pdf.savefig(fig)
        plt.close(fig)  # Prevents inline display in notebooks

print(f"[DONE] All plots saved to: {pdf_path}")


### Batch call with different max_time

In [ ]:
import numpy as np
import os
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt

pdf_path = f'{task}_radial_cdf_all_time.pdf'
with PdfPages(pdf_path) as pdf:
    for max_time in range(100, 1300, 100):
        print(f"[INFO] Analyzing max_time = {max_time} seconds")

        r_saline = get_batch_radial_distances(saline_id, conn, table='dlc_table', max_time=max_time)
        r_ghrelin = get_batch_radial_distances(ghrelin_id, conn, table='dlc_table', max_time=max_time)
        r_exc = get_batch_radial_distances(Exc_id, conn, table='dlc_table', max_time=max_time)
        r_inh = get_batch_radial_distances(Inh_id, conn, table='dlc_table', max_time=max_time)

        if len(r_saline) == 0 or len(r_ghrelin) == 0:
            print(f"[WARNING] Skipping max_time = {max_time} due to missing data")
            continue

        r_saline_all = np.concatenate(r_saline)
        r_ghrelin_all = np.concatenate(r_ghrelin)
        r_exc_all = np.concatenate(r_exc)
        r_inh_all = np.concatenate(r_inh)
        
        fig, ax = compare_distributions(
            r_saline_all, r_ghrelin_all, r_exc_all, r_inh_all, 
            kind='cdf', return_data=False
        )
        
        ax.legend(['Saline', 'Ghrelin', 'Excitatory', 'Inhibitory'])
        ax.set_title(f"{task} | max_time = {max_time}s")
        pdf.savefig(fig)
        plt.close(fig)  # Prevents inline display in notebooks

print(f"[DONE] All plots saved to: {pdf_path}")


### Best parameter call

In [ ]:
import numpy as np

r_saline = get_batch_radial_distances(saline_id, conn, table='dlc_table', center=None, max_time=None, radius_limit=0.7)
r_ghrelin = get_batch_radial_distances(ghrelin_id, conn, table='dlc_table', center=None, max_time=None, radius_limit=0.7)
r_exc = get_batch_radial_distances(Exc_id, conn, table='dlc_table', center=None, max_time=None, radius_limit=0.7)
r_inh = get_batch_radial_distances(Inh_id, conn, table='dlc_table', center=None, max_time=None, radius_limit=0.7)

# Flatten
r_saline_all = np.concatenate(r_saline)
r_ghrelin_all = np.concatenate(r_ghrelin)
r_exc_all = np.concatenate(r_exc)
r_inh_all = np.concatenate(r_inh)

# CDF + KS + AUC differences
fig, ax, result = compare_distributions(
    r_ghrelin_all, r_exc_all, r_inh_all,
    kind='cdf', return_data=True
)

ax.legend(['Ghrelin', 'Excitatory', 'Inhibitory'])
ax.set_title(f"Radial Distribution: {task}")

# fig.savefig(f"{task}_radial_pdf_comparison.pdf", dpi=300, bbox_inches='tight')


### Save outputs

In [ ]:
import numpy as np
import pandas as pd

heights, bin_edges = np.histogram(r_saline_all, bins=100, density=True)
bin_centers = bin_edges[:-1] + np.diff(bin_edges)/2

df = pd.DataFrame({'bin_center': bin_centers, 'height': heights})
df.to_csv(f'{task}_radial_histogram_saline.csv', index=False)

heights, bin_edges = np.histogram(r_ghrelin_all, bins=100, density=True)
bin_centers = bin_edges[:-1] + np.diff(bin_edges)/2

df = pd.DataFrame({'bin_center': bin_centers, 'height': heights})
df.to_csv(f'{task}_radial_histogram_ghrelin.csv', index=False)


### Batch call with different radius and time limit

In [ ]:
import numpy as np
import os
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt

pdf_path = f'{task}_radial_cdf_radius_time_grid.pdf'
with PdfPages(pdf_path) as pdf:
    for max_time in range(100, 1300, 300):
        for radius in np.arange(0.1, 1.7, 0.3):
            print(f"[INFO] Analyzing radius = {radius:.1f}, max_time = {max_time}")

            r_saline = get_batch_radial_distances(saline_id, conn, radius_limit=radius, max_time=max_time)
            r_ghrelin = get_batch_radial_distances(ghrelin_id, conn, radius_limit=radius, max_time=max_time)

            if len(r_saline) == 0 or len(r_ghrelin) == 0:
                print(f"[WARNING] Skipping radius = {radius:.1f}, max_time = {max_time} due to missing data")
                continue

            r_saline_all = np.concatenate(r_saline)
            r_ghrelin_all = np.concatenate(r_ghrelin)

            fig, ax = compare_distributions(
                r_saline_all, r_ghrelin_all,
                kind='cdf', return_data=False
            )
            
            ax.legend(['Saline', 'Ghrelin', 'Excitatory', 'Inhibitory'])
            ax.set_title(f"{task} | radius = {radius:.1f}, max_time = {max_time}")
            pdf.savefig(fig)
            plt.close(fig)

print(f"[DONE] All plots saved to: {pdf_path}")


In [ ]:
import importlib
import Python_scripts.Feature_functions.plot_radial_density_per_trial
importlib.reload(Python_scripts.Feature_functions.plot_radial_density_per_trial)
from Python_scripts.Feature_functions.plot_radial_density_per_trial import plot_radial_density_per_trial

In [ ]:
plot_radial_density_per_trial(ghrelin_id, conn)

In [ ]:
import pandas as pd

q = "SELECT id, video_name, genotype, task, health, name, date, maze_number, num_frames, frame_rate, video_width, video_height FROM dlc_table ORDER BY id"
# q = "SELECT * FROM dlc_table ORDER BY id LIMIT 1"

df = pd.read_sql_query(q, conn)
# print(df.columns)
df.to_csv('dlc_table_dump.csv', index=False)
